In [90]:
import pandas as pd 
import numpy as np

In [91]:
dataset = pd.read_csv('ratings.csv')
dataset.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [92]:
movieTitles = pd.read_csv('movies.csv')
movieTitles.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [93]:
dataset = pd.merge(dataset, movieTitles, on='movieId')
dataset.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [94]:
movieMatrix = dataset.pivot_table(index='userId', columns='movieId', values='rating')
movieMatrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [95]:
userIdMap = {}
tempInd = 0
for i in dataset['userId'].unique():
    userIdMap[i] = tempInd
    tempInd += 1

In [96]:
itemIdMap = {}
tempInd = 0
for i in dataset['movieId'].unique():
    itemIdMap[i] = tempInd
    tempInd += 1

In [97]:
train = pd.read_csv('train.csv')

In [98]:
movieMatrix = np.array(movieMatrix)

movieMatrix[:,:] = np.nan
for i in train.index:
    movieMatrix[userIdMap[train['userId'][i]], itemIdMap[train['movieId'][i]]] = train['rating'][i]
np.count_nonzero(~np.isnan(movieMatrix))

70829

In [99]:
test = pd.read_csv('test.csv')

testMatrix = np.empty((610, 9724))
testMatrix[:,:] = 0
for i in test.index:
    testMatrix[userIdMap[test['userId'][i]], itemIdMap[test['movieId'][i]]] = test['rating'][i]
testMatrix

array([[0., 4., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
movieMatrix.shape

In [101]:
numUsers, numItems = movieMatrix.shape

In [102]:
where_are_NaNs = np.isnan(movieMatrix)
where_are_NaNs

array([[False,  True, False, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       [False,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [103]:
rowMean = np.nanmean(movieMatrix, axis=1)

for j in range(0, numUsers):
    for k in range(0, numItems):
        if where_are_NaNs[j, k]:
            movieMatrix[j, k] = rowMean[j]

movieMatrix

array([[4.        , 4.33540373, 4.        , ..., 4.33540373, 4.33540373,
        4.33540373],
       [4.        , 3.71428571, 3.71428571, ..., 3.71428571, 3.71428571,
        3.71428571],
       [4.5       , 3.22357724, 3.22357724, ..., 3.22357724, 3.22357724,
        3.22357724],
       ...,
       [4.        , 4.        , 4.        , ..., 4.        , 4.        ,
        4.        ],
       [3.52631579, 3.52631579, 3.52631579, ..., 3.52631579, 3.52631579,
        3.52631579],
       [4.        , 4.        , 4.        , ..., 4.        , 4.        ,
        4.        ]])

In [104]:
R = movieMatrix
R

array([[4.        , 4.33540373, 4.        , ..., 4.33540373, 4.33540373,
        4.33540373],
       [4.        , 3.71428571, 3.71428571, ..., 3.71428571, 3.71428571,
        3.71428571],
       [4.5       , 3.22357724, 3.22357724, ..., 3.22357724, 3.22357724,
        3.22357724],
       ...,
       [4.        , 4.        , 4.        , ..., 4.        , 4.        ,
        4.        ],
       [3.52631579, 3.52631579, 3.52631579, ..., 3.52631579, 3.52631579,
        3.52631579],
       [4.        , 4.        , 4.        , ..., 4.        , 4.        ,
        4.        ]])

In [105]:
np.linalg.matrix_rank(R)

610

In [106]:
K = 400

In [107]:
U = np.random.randn(numUsers, K)
U

array([[ 1.15524191, -1.13201602,  0.37551666, ..., -1.21585967,
        -0.81954088, -0.26571181],
       [ 1.99223748, -0.97523848,  0.17357616, ..., -2.06747589,
        -0.69791366,  0.38527694],
       [ 0.20921705,  0.641629  ,  1.68328549, ..., -0.00688965,
         0.6349344 ,  1.56110856],
       ...,
       [ 1.58579423,  0.21702008, -1.17413072, ...,  0.17867463,
        -0.44123291, -0.71259649],
       [-0.81374965,  0.64325161, -1.29174981, ..., -1.41622021,
         1.02645823, -0.46813954],
       [-0.16606871, -0.04680985, -0.70701866, ...,  1.45076147,
        -0.10039647, -0.04976567]])

In [108]:
V = np.random.randn(numItems, K)
V

array([[-1.58156817, -0.70906278, -1.34450035, ...,  0.18356981,
         0.08840609, -1.65491584],
       [-0.72400781, -0.41776456,  0.46212898, ...,  0.75043786,
        -0.69605526, -0.84693731],
       [-1.26137745, -0.67966561,  0.02112106, ...,  0.82011119,
        -1.86828583, -0.85262564],
       ...,
       [ 1.13702496,  0.97591525, -0.11809038, ..., -0.09207888,
         0.44600833, -0.18220565],
       [-1.28961494,  0.63557307,  1.62656141, ...,  1.14473368,
        -0.80365457, -0.33791456],
       [ 1.01295576,  0.09752235, -1.02903585, ..., -0.81268602,
         0.01849921, -1.18714191]])

In [109]:
alpha = 0.000001

In [110]:
for i in range(20000):
    error = R - np.matmul(U, V.transpose())
    
    #Batch Update
    U = U + alpha * np.matmul(error, V)
    V = V + alpha * np.matmul(error.transpose(), U)
    
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  5434.234073191185
Iteration:  1  ---> Frobenius norm:  5375.724634958778
Iteration:  2  ---> Frobenius norm:  5318.134257642483
Iteration:  3  ---> Frobenius norm:  5261.441914736303
Iteration:  4  ---> Frobenius norm:  5205.627281608953
Iteration:  5  ---> Frobenius norm:  5150.670704837723
Iteration:  6  ---> Frobenius norm:  5096.553173170522
Iteration:  7  ---> Frobenius norm:  5043.256290014761
Iteration:  8  ---> Frobenius norm:  4990.762247358978
Iteration:  9  ---> Frobenius norm:  4939.05380103975
Iteration:  10  ---> Frobenius norm:  4888.114247272555
Iteration:  11  ---> Frobenius norm:  4837.927400370853
Iteration:  12  ---> Frobenius norm:  4788.477571582889
Iteration:  13  ---> Frobenius norm:  4739.749548980476
Iteration:  14  ---> Frobenius norm:  4691.728578338484
Iteration:  15  ---> Frobenius norm:  4644.4003449478
Iteration:  16  ---> Frobenius norm:  4597.750956308346
Iteration:  17  ---> Frobenius norm:  4551.766925652233
Itera

Iteration:  145  ---> Frobenius norm:  1657.8360417952636
Iteration:  146  ---> Frobenius norm:  1648.071355032636
Iteration:  147  ---> Frobenius norm:  1638.4130969155678
Iteration:  148  ---> Frobenius norm:  1628.8599707032859
Iteration:  149  ---> Frobenius norm:  1619.4106935402099
Iteration:  150  ---> Frobenius norm:  1610.0639962407913
Iteration:  151  ---> Frobenius norm:  1600.8186230798967
Iteration:  152  ---> Frobenius norm:  1591.6733315886015
Iteration:  153  ---> Frobenius norm:  1582.6268923552616
Iteration:  154  ---> Frobenius norm:  1573.6780888317357
Iteration:  155  ---> Frobenius norm:  1564.8257171446337
Iteration:  156  ---> Frobenius norm:  1556.0685859114628
Iteration:  157  ---> Frobenius norm:  1547.4055160615626
Iteration:  158  ---> Frobenius norm:  1538.8353406617018
Iteration:  159  ---> Frobenius norm:  1530.356904746228
Iteration:  160  ---> Frobenius norm:  1521.9690651516594
Iteration:  161  ---> Frobenius norm:  1513.6706903556033
Iteration:  162 

Iteration:  288  ---> Frobenius norm:  894.1819935385955
Iteration:  289  ---> Frobenius norm:  891.083896174882
Iteration:  290  ---> Frobenius norm:  887.9968540017211
Iteration:  291  ---> Frobenius norm:  884.9206567515089
Iteration:  292  ---> Frobenius norm:  881.8550988724833
Iteration:  293  ---> Frobenius norm:  878.7999794944865
Iteration:  294  ---> Frobenius norm:  875.7551023944233
Iteration:  295  ---> Frobenius norm:  872.720275961412
Iteration:  296  ---> Frobenius norm:  869.6953131616232
Iteration:  297  ---> Frobenius norm:  866.6800315028047
Iteration:  298  ---> Frobenius norm:  863.6742529984881
Iteration:  299  ---> Frobenius norm:  860.6778041318754
Iteration:  300  ---> Frobenius norm:  857.6905158194027
Iteration:  301  ---> Frobenius norm:  854.7122233739786
Iteration:  302  ---> Frobenius norm:  851.7427664678967
Iteration:  303  ---> Frobenius norm:  848.7819890954178
Iteration:  304  ---> Frobenius norm:  845.8297395350239
Iteration:  305  ---> Frobenius n

Iteration:  433  ---> Frobenius norm:  518.7683724800869
Iteration:  434  ---> Frobenius norm:  516.6928035794983
Iteration:  435  ---> Frobenius norm:  514.6261145006413
Iteration:  436  ---> Frobenius norm:  512.5683422427156
Iteration:  437  ---> Frobenius norm:  510.5195230437933
Iteration:  438  ---> Frobenius norm:  508.47969237005833
Iteration:  439  ---> Frobenius norm:  506.44888490559265
Iteration:  440  ---> Frobenius norm:  504.4271345427047
Iteration:  441  ---> Frobenius norm:  502.41447437279214
Iteration:  442  ---> Frobenius norm:  500.410936677731
Iteration:  443  ---> Frobenius norm:  498.4165529217879
Iteration:  444  ---> Frobenius norm:  496.43135374404443
Iteration:  445  ---> Frobenius norm:  494.45536895132807
Iteration:  446  ---> Frobenius norm:  492.4886275116431
Iteration:  447  ---> Frobenius norm:  490.5311575480917
Iteration:  448  ---> Frobenius norm:  488.58298633328036
Iteration:  449  ---> Frobenius norm:  486.64414028420055
Iteration:  450  ---> Fro

Iteration:  576  ---> Frobenius norm:  316.3101615434794
Iteration:  577  ---> Frobenius norm:  315.5098268949613
Iteration:  578  ---> Frobenius norm:  314.71644518123355
Iteration:  579  ---> Frobenius norm:  313.9299741437677
Iteration:  580  ---> Frobenius norm:  313.1503714817449
Iteration:  581  ---> Frobenius norm:  312.3775948577219
Iteration:  582  ---> Frobenius norm:  311.6116019032238
Iteration:  583  ---> Frobenius norm:  310.8523502242625
Iteration:  584  ---> Frobenius norm:  310.09979740678233
Iteration:  585  ---> Frobenius norm:  309.3539010220312
Iteration:  586  ---> Frobenius norm:  308.61461863185923
Iteration:  587  ---> Frobenius norm:  307.8819077939434
Iteration:  588  ---> Frobenius norm:  307.1557260669393
Iteration:  589  ---> Frobenius norm:  306.43603101555897
Iteration:  590  ---> Frobenius norm:  305.72278021557725
Iteration:  591  ---> Frobenius norm:  305.015931258763
Iteration:  592  ---> Frobenius norm:  304.31544175773934
Iteration:  593  ---> Frob

...

Iteration:  1424  ---> Frobenius norm:  236.11560659214933
Iteration:  1425  ---> Frobenius norm:  236.11550639079377
Iteration:  1426  ---> Frobenius norm:  236.11540708525132
Iteration:  1427  ---> Frobenius norm:  236.11530866662636
Iteration:  1428  ---> Frobenius norm:  236.1152111261177
Iteration:  1429  ---> Frobenius norm:  236.11511445501716
Iteration:  1430  ---> Frobenius norm:  236.11501864470898
Iteration:  1431  ---> Frobenius norm:  236.11492368666853
Iteration:  1432  ---> Frobenius norm:  236.11482957246133
Iteration:  1433  ---> Frobenius norm:  236.11473629374217
Iteration:  1434  ---> Frobenius norm:  236.114643842254
Iteration:  1435  ---> Frobenius norm:  236.11455220982705
Iteration:  1436  ---> Frobenius norm:  236.11446138837786
Iteration:  1437  ---> Frobenius norm:  236.11437136990827
Iteration:  1438  ---> Frobenius norm:  236.11428214650454
Iteration:  1439  ---> Frobenius norm:  236.11419371033637
Iteration:  1440  ---> Frobenius norm:  236.11410605365606


Iteration:  1564  ---> Frobenius norm:  236.10741492480398
Iteration:  1565  ---> Frobenius norm:  236.10738371040873
Iteration:  1566  ---> Frobenius norm:  236.10735273420832
Iteration:  1567  ---> Frobenius norm:  236.10732199409273
Iteration:  1568  ---> Frobenius norm:  236.10729148797262
Iteration:  1569  ---> Frobenius norm:  236.10726121377905
Iteration:  1570  ---> Frobenius norm:  236.1072311694634
Iteration:  1571  ---> Frobenius norm:  236.10720135299692
Iteration:  1572  ---> Frobenius norm:  236.10717176237077
Iteration:  1573  ---> Frobenius norm:  236.10714239559564
Iteration:  1574  ---> Frobenius norm:  236.10711325070156
Iteration:  1575  ---> Frobenius norm:  236.10708432573787
Iteration:  1576  ---> Frobenius norm:  236.10705561877282
Iteration:  1577  ---> Frobenius norm:  236.10702712789333
Iteration:  1578  ---> Frobenius norm:  236.1069988512051
Iteration:  1579  ---> Frobenius norm:  236.10697078683214
Iteration:  1580  ---> Frobenius norm:  236.10694293291664

Iteration:  1704  ---> Frobenius norm:  236.10466258657792
Iteration:  1705  ---> Frobenius norm:  236.10465089438915
Iteration:  1706  ---> Frobenius norm:  236.10463927651128
Iteration:  1707  ---> Frobenius norm:  236.1046277323703
Iteration:  1708  ---> Frobenius norm:  236.10461626139724
Iteration:  1709  ---> Frobenius norm:  236.10460486302804
Iteration:  1710  ---> Frobenius norm:  236.10459353670356
Iteration:  1711  ---> Frobenius norm:  236.1045822818694
Iteration:  1712  ---> Frobenius norm:  236.1045710979761
Iteration:  1713  ---> Frobenius norm:  236.1045599844789
Iteration:  1714  ---> Frobenius norm:  236.1045489408377
Iteration:  1715  ---> Frobenius norm:  236.10453796651706
Iteration:  1716  ---> Frobenius norm:  236.10452706098624
Iteration:  1717  ---> Frobenius norm:  236.10451622371897
Iteration:  1718  ---> Frobenius norm:  236.1045054541936
Iteration:  1719  ---> Frobenius norm:  236.10449475189287
Iteration:  1720  ---> Frobenius norm:  236.10448411630404
Ite

Iteration:  1844  ---> Frobenius norm:  236.10355208033337
Iteration:  1845  ---> Frobenius norm:  236.10354686810754
Iteration:  1846  ---> Frobenius norm:  236.10354168269396
Iteration:  1847  ---> Frobenius norm:  236.1035365239093
Iteration:  1848  ---> Frobenius norm:  236.10353139157164
Iteration:  1849  ---> Frobenius norm:  236.1035262855004
Iteration:  1850  ---> Frobenius norm:  236.10352120551647
Iteration:  1851  ---> Frobenius norm:  236.10351615144194
Iteration:  1852  ---> Frobenius norm:  236.1035111231004
Iteration:  1853  ---> Frobenius norm:  236.10350612031664
Iteration:  1854  ---> Frobenius norm:  236.10350114291683


KeyboardInterrupt: 

In [111]:
finalFilledMatrix = np.matmul(U, V.transpose())
finalFilledMatrix

array([[4.4113449 , 4.34884719, 4.34662573, ..., 4.32751768, 4.3528341 ,
        4.32933645],
       [3.82029975, 3.71087048, 3.75383448, ..., 3.70327811, 3.71009535,
        3.71011314],
       [3.38024504, 3.17652213, 3.22012517, ..., 3.20265768, 3.16945254,
        3.24298804],
       ...,
       [4.08279763, 3.99485343, 4.04583339, ..., 4.00794301, 3.99825281,
        3.99013238],
       [3.61205024, 3.53502536, 3.55832607, ..., 3.5462203 , 3.50757325,
        3.52874855],
       [4.10337925, 3.9780307 , 4.03369594, ..., 4.0131287 , 3.99702314,
        3.98600728]])

In [112]:
errorBatch = testMatrix-finalFilledMatrix
errorBatch[where_are_NaNs] = 0
np.linalg.norm(errorBatch)

943.3604507103306

## Regularization

In [113]:
UReg =  np.random.randn(numUsers, K)
UReg

array([[-2.00528618e-01, -1.53579816e+00, -3.92824541e-04, ...,
        -2.58266618e-01, -3.15741200e-01,  1.17037534e+00],
       [-2.72494227e-01, -1.67935131e+00, -5.78328066e-01, ...,
         7.87617847e-01, -4.35848787e-01,  2.55067200e-01],
       [ 1.25098186e-01, -3.53802081e-01,  5.57010771e-01, ...,
        -1.12671977e+00,  3.32074744e-01,  1.63782910e+00],
       ...,
       [-5.01646703e-01, -3.13948495e-01, -5.55652063e-01, ...,
         1.03735261e+00, -6.10987670e-01, -1.17973678e+00],
       [ 1.54252982e+00,  9.36796059e-01,  5.29384126e-01, ...,
         3.39997404e-01, -2.16134015e+00, -2.26434113e-01],
       [-8.17108392e-01,  5.42150570e-01, -7.80119838e-01, ...,
         1.37338236e+00, -4.94161083e-01, -1.92755768e+00]])

In [114]:
VReg = np.random.randn(numItems, K)
VReg

array([[ 1.10529868,  1.54070837,  0.07008154, ..., -0.10156106,
         0.58625575,  0.00733928],
       [ 0.58958109, -0.12842629, -1.33567306, ..., -0.95454238,
        -0.40762383,  0.51368489],
       [ 1.74486857, -0.4824154 ,  1.37767993, ..., -0.164254  ,
         0.70057157,  0.53688437],
       ...,
       [ 0.67841375,  0.60263922,  0.29217997, ..., -0.44656032,
         0.50573418,  0.05972433],
       [ 0.15535074,  0.13592497,  0.66961932, ...,  0.86303304,
        -0.36548004,  0.47964416],
       [-1.72505541, -0.02758504,  2.50333035, ...,  0.27607251,
         2.87715316, -0.73446801]])

In [115]:
regularizationFactor = 100

In [116]:
for i in range(10000):
    error = R - np.matmul(UReg, VReg.transpose())
    
    #Batch Update
    UReg = (1 - alpha*regularizationFactor) * UReg + alpha * np.matmul(error, VReg)
    VReg = (1 - alpha*regularizationFactor) * VReg + alpha * np.matmul(error.transpose(), UReg)
    
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  5361.35913273706
Iteration:  1  ---> Frobenius norm:  5302.634861110975
Iteration:  2  ---> Frobenius norm:  5244.851637614435
Iteration:  3  ---> Frobenius norm:  5187.987696650605
Iteration:  4  ---> Frobenius norm:  5132.022002249
Iteration:  5  ---> Frobenius norm:  5076.934216020543
Iteration:  6  ---> Frobenius norm:  5022.704666824732
Iteration:  7  ---> Frobenius norm:  4969.314322041648
Iteration:  8  ---> Frobenius norm:  4916.744760349308
Iteration:  9  ---> Frobenius norm:  4864.97814591389
Iteration:  10  ---> Frobenius norm:  4813.9972039068425
Iteration:  11  ---> Frobenius norm:  4763.78519726894
Iteration:  12  ---> Frobenius norm:  4714.3259046468065
Iteration:  13  ---> Frobenius norm:  4665.603599432534
Iteration:  14  ---> Frobenius norm:  4617.60302984175
Iteration:  15  ---> Frobenius norm:  4570.309399969766
Iteration:  16  ---> Frobenius norm:  4523.708351769537
Iteration:  17  ---> Frobenius norm:  4477.78594789879
Iteratio

Iteration:  145  ---> Frobenius norm:  1622.695784830083
Iteration:  146  ---> Frobenius norm:  1613.1762876960493
Iteration:  147  ---> Frobenius norm:  1603.7601966853715
Iteration:  148  ---> Frobenius norm:  1594.4461996302753
Iteration:  149  ---> Frobenius norm:  1585.232998933846
Iteration:  150  ---> Frobenius norm:  1576.1193113494958
Iteration:  151  ---> Frobenius norm:  1567.1038677660806
Iteration:  152  ---> Frobenius norm:  1558.185412998537
Iteration:  153  ---> Frobenius norm:  1549.362705583888
Iteration:  154  ---> Frobenius norm:  1540.6345175824913
Iteration:  155  ---> Frobenius norm:  1531.9996343843957
Iteration:  156  ---> Frobenius norm:  1523.4568545206773
Iteration:  157  ---> Frobenius norm:  1515.004989479626
Iteration:  158  ---> Frobenius norm:  1506.6428635276695
Iteration:  159  ---> Frobenius norm:  1498.3693135349022
Iteration:  160  ---> Frobenius norm:  1490.1831888051145
Iteration:  161  ---> Frobenius norm:  1482.0833509101983
Iteration:  162  --

Iteration:  288  ---> Frobenius norm:  865.377345342799
Iteration:  289  ---> Frobenius norm:  862.1863602084909
Iteration:  290  ---> Frobenius norm:  859.0059742559013
Iteration:  291  ---> Frobenius norm:  855.8360124977198
Iteration:  292  ---> Frobenius norm:  852.6763046313141
Iteration:  293  ---> Frobenius norm:  849.5266849971607
Iteration:  294  ---> Frobenius norm:  846.3869925369129
Iteration:  295  ---> Frobenius norm:  843.2570707511048
Iteration:  296  ---> Frobenius norm:  840.1367676564847
Iteration:  297  ---> Frobenius norm:  837.0259357429821
Iteration:  298  ---> Frobenius norm:  833.9244319303049
Iteration:  299  ---> Frobenius norm:  830.8321175241641
Iteration:  300  ---> Frobenius norm:  827.7488581721324
Iteration:  301  ---> Frobenius norm:  824.6745238191289
Iteration:  302  ---> Frobenius norm:  821.608988662537
Iteration:  303  ---> Frobenius norm:  818.5521311069539
Iteration:  304  ---> Frobenius norm:  815.503833718574
Iteration:  305  ---> Frobenius no

Iteration:  432  ---> Frobenius norm:  487.1967739315106
Iteration:  433  ---> Frobenius norm:  485.1861908927084
Iteration:  434  ---> Frobenius norm:  483.1858484561621
Iteration:  435  ---> Frobenius norm:  481.1957695741158
Iteration:  436  ---> Frobenius norm:  479.21597626107155
Iteration:  437  ---> Frobenius norm:  477.2464895896313
Iteration:  438  ---> Frobenius norm:  475.2873296868598
Iteration:  439  ---> Frobenius norm:  473.338515731157
Iteration:  440  ---> Frobenius norm:  471.4000659496318
Iteration:  441  ---> Frobenius norm:  469.47199761596625
Iteration:  442  ---> Frobenius norm:  467.55432704875966
Iteration:  443  ---> Frobenius norm:  465.6470696103431
Iteration:  444  ---> Frobenius norm:  463.7502397060538
Iteration:  445  ---> Frobenius norm:  461.8638507839579
Iteration:  446  ---> Frobenius norm:  459.9879153350143
Iteration:  447  ---> Frobenius norm:  458.12244489366503
Iteration:  448  ---> Frobenius norm:  456.2674500388451
Iteration:  449  ---> Froben

Iteration:  575  ---> Frobenius norm:  300.5748149983869
Iteration:  576  ---> Frobenius norm:  299.8867337091956
Iteration:  577  ---> Frobenius norm:  299.20518678865557
Iteration:  578  ---> Frobenius norm:  298.53012743272404
Iteration:  579  ---> Frobenius norm:  297.8615089087756
Iteration:  580  ---> Frobenius norm:  297.1992845604048
Iteration:  581  ---> Frobenius norm:  296.543407812147
Iteration:  582  ---> Frobenius norm:  295.8938321741147
Iteration:  583  ---> Frobenius norm:  295.25051124655346
Iteration:  584  ---> Frobenius norm:  294.6133987243139
Iteration:  585  ---> Frobenius norm:  293.98244840124306
Iteration:  586  ---> Frobenius norm:  293.3576141744941
Iteration:  587  ---> Frobenius norm:  292.7388500487554
Iteration:  588  ---> Frobenius norm:  292.1261101403984
Iteration:  589  ---> Frobenius norm:  291.5193486815461
Iteration:  590  ---> Frobenius norm:  290.91852002406137
Iteration:  591  ---> Frobenius norm:  290.3235786434561
Iteration:  592  ---> Frobe

...

Iteration:  1562  ---> Frobenius norm:  236.46483874398996
Iteration:  1563  ---> Frobenius norm:  236.4649029943801
Iteration:  1564  ---> Frobenius norm:  236.46496725040114
Iteration:  1565  ---> Frobenius norm:  236.46503151167897
Iteration:  1566  ---> Frobenius norm:  236.46509577784673
Iteration:  1567  ---> Frobenius norm:  236.4651600485443
Iteration:  1568  ---> Frobenius norm:  236.46522432341834
Iteration:  1569  ---> Frobenius norm:  236.46528860212237
Iteration:  1570  ---> Frobenius norm:  236.46535288431636
Iteration:  1571  ---> Frobenius norm:  236.4654171696669
Iteration:  1572  ---> Frobenius norm:  236.46548145784686
Iteration:  1573  ---> Frobenius norm:  236.4655457485355
Iteration:  1574  ---> Frobenius norm:  236.46561004141844
Iteration:  1575  ---> Frobenius norm:  236.46567433618722
Iteration:  1576  ---> Frobenius norm:  236.46573863253963
Iteration:  1577  ---> Frobenius norm:  236.46580293017928
Iteration:  1578  ---> Frobenius norm:  236.4658672288157
It

Iteration:  1702  ---> Frobenius norm:  236.47379660319743
Iteration:  1703  ---> Frobenius norm:  236.47386002333351
Iteration:  1704  ---> Frobenius norm:  236.47392343479848
Iteration:  1705  ---> Frobenius norm:  236.47398683762407
Iteration:  1706  ---> Frobenius norm:  236.47405023184274
Iteration:  1707  ---> Frobenius norm:  236.47411361748752
Iteration:  1708  ---> Frobenius norm:  236.47417699459214
Iteration:  1709  ---> Frobenius norm:  236.47424036319094
Iteration:  1710  ---> Frobenius norm:  236.47430372331885
Iteration:  1711  ---> Frobenius norm:  236.4743670750114
Iteration:  1712  ---> Frobenius norm:  236.47443041830476
Iteration:  1713  ---> Frobenius norm:  236.47449375323555
Iteration:  1714  ---> Frobenius norm:  236.47455707984108
Iteration:  1715  ---> Frobenius norm:  236.47462039815912
Iteration:  1716  ---> Frobenius norm:  236.474683708228
Iteration:  1717  ---> Frobenius norm:  236.47474701008647
Iteration:  1718  ---> Frobenius norm:  236.47481030377406


Iteration:  1842  ---> Frobenius norm:  236.48261105882935
Iteration:  1843  ---> Frobenius norm:  236.48267371927082
Iteration:  1844  ---> Frobenius norm:  236.48273637789026
Iteration:  1845  ---> Frobenius norm:  236.48279903473636
Iteration:  1846  ---> Frobenius norm:  236.48286168985786
Iteration:  1847  ---> Frobenius norm:  236.48292434330318
Iteration:  1848  ---> Frobenius norm:  236.48298699512074
Iteration:  1849  ---> Frobenius norm:  236.48304964535865
Iteration:  1850  ---> Frobenius norm:  236.4831122940651
Iteration:  1851  ---> Frobenius norm:  236.48317494128804
Iteration:  1852  ---> Frobenius norm:  236.48323758707528
Iteration:  1853  ---> Frobenius norm:  236.48330023147446
Iteration:  1854  ---> Frobenius norm:  236.48336287453319
Iteration:  1855  ---> Frobenius norm:  236.48342551629875
Iteration:  1856  ---> Frobenius norm:  236.48348815681842
Iteration:  1857  ---> Frobenius norm:  236.48355079613944
Iteration:  1858  ---> Frobenius norm:  236.4836134343086

Iteration:  1982  ---> Frobenius norm:  236.4913853551392
Iteration:  1983  ---> Frobenius norm:  236.4914481683261
Iteration:  1984  ---> Frobenius norm:  236.49151098506096
Iteration:  1985  ---> Frobenius norm:  236.4915738053717
Iteration:  1986  ---> Frobenius norm:  236.49163662928618
Iteration:  1987  ---> Frobenius norm:  236.49169945683224
Iteration:  1988  ---> Frobenius norm:  236.49176228803745
Iteration:  1989  ---> Frobenius norm:  236.49182512292933
Iteration:  1990  ---> Frobenius norm:  236.49188796153524
Iteration:  1991  ---> Frobenius norm:  236.4919508038824
Iteration:  1992  ---> Frobenius norm:  236.49201364999794
Iteration:  1993  ---> Frobenius norm:  236.49207649990882
Iteration:  1994  ---> Frobenius norm:  236.4921393536419


KeyboardInterrupt: 

In [117]:
finalFilledMatrixReg = np.matmul(UReg, VReg.transpose())
finalFilledMatrixReg

array([[4.36347783, 4.2913621 , 4.30471387, ..., 4.29436367, 4.26785348,
        4.33260115],
       [3.7703884 , 3.65332704, 3.72647045, ..., 3.66553905, 3.69524304,
        3.68613215],
       [3.33448291, 3.20903809, 3.2517325 , ..., 3.17164754, 3.16159683,
        3.13704707],
       ...,
       [4.05291864, 3.94339708, 4.00601196, ..., 3.96119382, 3.95977551,
        3.95630979],
       [3.57360742, 3.46794431, 3.52996138, ..., 3.49563296, 3.4867464 ,
        3.50152467],
       [4.05007483, 3.94374221, 4.01314083, ..., 3.95612293, 3.96582868,
        3.95779238]])

In [118]:
errorBatchReg = testMatrix-finalFilledMatrixReg
errorBatchReg[where_are_NaNs] = 0
np.linalg.norm(errorBatchReg)

933.4572768164145

## Biases included

In [119]:
UBias =  np.random.randn(numUsers, K+2)
UBias[:,K+1] = 1
UBias

array([[ 1.4280408 ,  0.22153332,  0.44760995, ...,  0.80510228,
        -0.1046513 ,  1.        ],
       [-0.39880256,  1.17848975, -0.54715279, ..., -1.74620232,
        -0.16498258,  1.        ],
       [ 0.10104089,  0.22601267, -0.95538724, ..., -0.59976029,
         1.29889002,  1.        ],
       ...,
       [ 1.8892558 ,  0.56621344,  0.77768061, ..., -1.45059783,
        -0.79900311,  1.        ],
       [ 0.67886719, -1.56383739,  1.31674801, ...,  1.73953201,
         3.05568582,  1.        ],
       [-1.24997455, -0.04906214,  0.51448718, ..., -0.28809394,
         1.37383224,  1.        ]])

In [120]:
VBias = np.random.randn(numItems, K+2)
VBias[:, K] = 1
VBias

array([[-0.27691394, -0.41555721,  1.10958798, ...,  0.36637843,
         1.        , -0.32201463],
       [-0.74417956, -1.64633299, -0.64949251, ...,  0.59122976,
         1.        ,  1.08220077],
       [ 2.50389294,  0.61554648, -0.83442554, ..., -1.62660005,
         1.        , -1.17144298],
       ...,
       [-0.04183051,  0.21684361,  0.5366486 , ...,  0.80790826,
         1.        ,  0.78755043],
       [-0.48552658,  0.45475809, -0.55126753, ...,  0.94201729,
         1.        , -1.68268646],
       [-0.55645527,  0.58338776,  2.2180149 , ..., -0.02252728,
         1.        ,  0.6624667 ]])

In [121]:
for i in range(2000):
    error = R - np.matmul(UBias, VBias.transpose())
    
    #Batch Update
    UBias = (1 - alpha*regularizationFactor) * UBias + alpha * np.matmul(error, VBias)
    VBias = (1 - alpha*regularizationFactor) * VBias + alpha * np.matmul(error.transpose(), UBias)
    UBias[:, K+1] = 1
    VBias[:, K] = 1
    
    # Calculating norm
    error[where_are_NaNs] = 0
    print("Iteration: ", i, " ---> Frobenius norm: ", np.linalg.norm(error))

Iteration:  0  ---> Frobenius norm:  5471.2567746789855
Iteration:  1  ---> Frobenius norm:  5409.6221052515575
Iteration:  2  ---> Frobenius norm:  5348.951954783822
Iteration:  3  ---> Frobenius norm:  5289.223919306747
Iteration:  4  ---> Frobenius norm:  5230.416350352581
Iteration:  5  ---> Frobenius norm:  5172.508321447418
Iteration:  6  ---> Frobenius norm:  5115.479596415355
Iteration:  7  ---> Frobenius norm:  5059.310599379493
Iteration:  8  ---> Frobenius norm:  5003.982386353419
Iteration:  9  ---> Frobenius norm:  4949.4766183244155
Iteration:  10  ---> Frobenius norm:  4895.775535736632
Iteration:  11  ---> Frobenius norm:  4842.861934288981
Iteration:  12  ---> Frobenius norm:  4790.7191419683895
Iteration:  13  ---> Frobenius norm:  4739.33099724458
Iteration:  14  ---> Frobenius norm:  4688.681828357568
Iteration:  15  ---> Frobenius norm:  4638.7564336337355
Iteration:  16  ---> Frobenius norm:  4589.540062770676
Iteration:  17  ---> Frobenius norm:  4541.01839903492

Iteration:  145  ---> Frobenius norm:  1422.7731813307266
Iteration:  146  ---> Frobenius norm:  1411.5878412681982
Iteration:  147  ---> Frobenius norm:  1400.5134396694316
Iteration:  148  ---> Frobenius norm:  1389.5487676442638
Iteration:  149  ---> Frobenius norm:  1378.6926322569022
Iteration:  150  ---> Frobenius norm:  1367.943856267222
Iteration:  151  ---> Frobenius norm:  1357.3012778772386
Iteration:  152  ---> Frobenius norm:  1346.7637504826237
Iteration:  153  ---> Frobenius norm:  1336.3301424291362
Iteration:  154  ---> Frobenius norm:  1325.99933677385
Iteration:  155  ---> Frobenius norm:  1315.770231051052
Iteration:  156  ---> Frobenius norm:  1305.6417370427025
Iteration:  157  ---> Frobenius norm:  1295.612780553341
Iteration:  158  ---> Frobenius norm:  1285.6823011893302
Iteration:  159  ---> Frobenius norm:  1275.8492521423323
Iteration:  160  ---> Frobenius norm:  1266.1125999769135
Iteration:  161  ---> Frobenius norm:  1256.4713244221825
Iteration:  162  --

Iteration:  289  ---> Frobenius norm:  553.5532003589336
Iteration:  290  ---> Frobenius norm:  550.8063348505646
Iteration:  291  ---> Frobenius norm:  548.0867049549439
Iteration:  292  ---> Frobenius norm:  545.3940607310569
Iteration:  293  ---> Frobenius norm:  542.728154425605
Iteration:  294  ---> Frobenius norm:  540.0887404480885
Iteration:  295  ---> Frobenius norm:  537.4755753463018
Iteration:  296  ---> Frobenius norm:  534.8884177822296
Iteration:  297  ---> Frobenius norm:  532.3270285083436
Iteration:  298  ---> Frobenius norm:  529.7911703442918
Iteration:  299  ---> Frobenius norm:  527.2806081539741
Iteration:  300  ---> Frobenius norm:  524.7951088230029
Iteration:  301  ---> Frobenius norm:  522.3344412365386
Iteration:  302  ---> Frobenius norm:  519.8983762574985
Iteration:  303  ---> Frobenius norm:  517.4866867051318
Iteration:  304  ---> Frobenius norm:  515.099147333956
Iteration:  305  ---> Frobenius norm:  512.7355348130511
Iteration:  306  ---> Frobenius n

Iteration:  432  ---> Frobenius norm:  346.3114310238996
Iteration:  433  ---> Frobenius norm:  345.68743954096146
Iteration:  434  ---> Frobenius norm:  345.0698889948671
Iteration:  435  ---> Frobenius norm:  344.45870784256357
Iteration:  436  ---> Frobenius norm:  343.8538252563198
Iteration:  437  ---> Frobenius norm:  343.2551711184068
Iteration:  438  ---> Frobenius norm:  342.662676015789
Iteration:  439  ---> Frobenius norm:  342.07627123482655
Iteration:  440  ---> Frobenius norm:  341.49588875598914
Iteration:  441  ---> Frobenius norm:  340.9214612485817
Iteration:  442  ---> Frobenius norm:  340.3529220654808
Iteration:  443  ---> Frobenius norm:  339.79020523788444
Iteration:  444  ---> Frobenius norm:  339.2332454700727
Iteration:  445  ---> Frobenius norm:  338.6819781341816
Iteration:  446  ---> Frobenius norm:  338.1363392649892
Iteration:  447  ---> Frobenius norm:  337.5962655547149
Iteration:  448  ---> Frobenius norm:  337.06169434783163
Iteration:  449  ---> Frob

Iteration:  575  ---> Frobenius norm:  297.98359094070423
Iteration:  576  ---> Frobenius norm:  297.8145914288526
Iteration:  577  ---> Frobenius norm:  297.64682588370533
Iteration:  578  ---> Frobenius norm:  297.48028028246284
Iteration:  579  ---> Frobenius norm:  297.31494077210505
Iteration:  580  ---> Frobenius norm:  297.1507936673846
Iteration:  581  ---> Frobenius norm:  296.9878254488429
Iteration:  582  ---> Frobenius norm:  296.8260227608462
Iteration:  583  ---> Frobenius norm:  296.6653724096434
Iteration:  584  ---> Frobenius norm:  296.5058613614454
Iteration:  585  ---> Frobenius norm:  296.3474767405242
Iteration:  586  ---> Frobenius norm:  296.19020582733344
Iteration:  587  ---> Frobenius norm:  296.0340360566494
Iteration:  588  ---> Frobenius norm:  295.8789550157318
Iteration:  589  ---> Frobenius norm:  295.7249504425054
Iteration:  590  ---> Frobenius norm:  295.5720102237611
Iteration:  591  ---> Frobenius norm:  295.42012239337726
Iteration:  592  ---> Fro

...

Iteration:  1426  ---> Frobenius norm:  251.89419265727602
Iteration:  1427  ---> Frobenius norm:  251.8707909806694
Iteration:  1428  ---> Frobenius norm:  251.84742351440366
Iteration:  1429  ---> Frobenius norm:  251.82409021235247
Iteration:  1430  ---> Frobenius norm:  251.80079102844604
Iteration:  1431  ---> Frobenius norm:  251.77752591667107
Iteration:  1432  ---> Frobenius norm:  251.75429483107035
Iteration:  1433  ---> Frobenius norm:  251.73109772574304
Iteration:  1434  ---> Frobenius norm:  251.70793455484434
Iteration:  1435  ---> Frobenius norm:  251.68480527258532
Iteration:  1436  ---> Frobenius norm:  251.66170983323312
Iteration:  1437  ---> Frobenius norm:  251.63864819111046
Iteration:  1438  ---> Frobenius norm:  251.6156203005958
Iteration:  1439  ---> Frobenius norm:  251.59262611612317
Iteration:  1440  ---> Frobenius norm:  251.56966559218196
Iteration:  1441  ---> Frobenius norm:  251.54673868331705
Iteration:  1442  ---> Frobenius norm:  251.52384534412846

Iteration:  1566  ---> Frobenius norm:  248.9310066817847
Iteration:  1567  ---> Frobenius norm:  248.91196954785323
Iteration:  1568  ---> Frobenius norm:  248.8929606776126
Iteration:  1569  ---> Frobenius norm:  248.87398003206144
Iteration:  1570  ---> Frobenius norm:  248.85502757224492
Iteration:  1571  ---> Frobenius norm:  248.83610325925488
Iteration:  1572  ---> Frobenius norm:  248.8172070542296
Iteration:  1573  ---> Frobenius norm:  248.79833891835395
Iteration:  1574  ---> Frobenius norm:  248.77949881285906
Iteration:  1575  ---> Frobenius norm:  248.76068669902241
Iteration:  1576  ---> Frobenius norm:  248.74190253816806
Iteration:  1577  ---> Frobenius norm:  248.7231462916661
Iteration:  1578  ---> Frobenius norm:  248.70441792093294
Iteration:  1579  ---> Frobenius norm:  248.68571738743117
Iteration:  1580  ---> Frobenius norm:  248.6670446526695
Iteration:  1581  ---> Frobenius norm:  248.64839967820276
Iteration:  1582  ---> Frobenius norm:  248.6297824256318
Ite

Iteration:  1706  ---> Frobenius norm:  246.52405328059524
Iteration:  1707  ---> Frobenius norm:  246.50861376404671
Iteration:  1708  ---> Frobenius norm:  246.49319748503353
Iteration:  1709  ---> Frobenius norm:  246.47780441066627
Iteration:  1710  ---> Frobenius norm:  246.4624345080963
Iteration:  1711  ---> Frobenius norm:  246.44708774451573
Iteration:  1712  ---> Frobenius norm:  246.43176408715786
Iteration:  1713  ---> Frobenius norm:  246.4164635032963
Iteration:  1714  ---> Frobenius norm:  246.4011859602458
Iteration:  1715  ---> Frobenius norm:  246.3859314253616
Iteration:  1716  ---> Frobenius norm:  246.37069986603964
Iteration:  1717  ---> Frobenius norm:  246.3554912497166
Iteration:  1718  ---> Frobenius norm:  246.34030554386973
Iteration:  1719  ---> Frobenius norm:  246.32514271601676
Iteration:  1720  ---> Frobenius norm:  246.31000273371603
Iteration:  1721  ---> Frobenius norm:  246.29488556456633
Iteration:  1722  ---> Frobenius norm:  246.27979117620694
It

Iteration:  1846  ---> Frobenius norm:  244.57451315465903
Iteration:  1847  ---> Frobenius norm:  244.56202462224874
Iteration:  1848  ---> Frobenius norm:  244.54955510444708
Iteration:  1849  ---> Frobenius norm:  244.53710457373677
Iteration:  1850  ---> Frobenius norm:  244.52467300263623
Iteration:  1851  ---> Frobenius norm:  244.51226036369985
Iteration:  1852  ---> Frobenius norm:  244.49986662951767
Iteration:  1853  ---> Frobenius norm:  244.48749177271563
Iteration:  1854  ---> Frobenius norm:  244.47513576595514
Iteration:  1855  ---> Frobenius norm:  244.46279858193347
Iteration:  1856  ---> Frobenius norm:  244.4504801933835
Iteration:  1857  ---> Frobenius norm:  244.43818057307362
Iteration:  1858  ---> Frobenius norm:  244.42589969380793
Iteration:  1859  ---> Frobenius norm:  244.4136375284259
Iteration:  1860  ---> Frobenius norm:  244.4013940498026
Iteration:  1861  ---> Frobenius norm:  244.38916923084852
Iteration:  1862  ---> Frobenius norm:  244.3769630445096
I

Iteration:  1986  ---> Frobenius norm:  242.99934940843247
Iteration:  1987  ---> Frobenius norm:  242.9892706834333
Iteration:  1988  ---> Frobenius norm:  242.97920745370948
Iteration:  1989  ---> Frobenius norm:  242.96915969642203
Iteration:  1990  ---> Frobenius norm:  242.9591273887629
Iteration:  1991  ---> Frobenius norm:  242.94911050795514
Iteration:  1992  ---> Frobenius norm:  242.93910903125254
Iteration:  1993  ---> Frobenius norm:  242.92912293593986
Iteration:  1994  ---> Frobenius norm:  242.91915219933273
Iteration:  1995  ---> Frobenius norm:  242.90919679877746
Iteration:  1996  ---> Frobenius norm:  242.89925671165136
Iteration:  1997  ---> Frobenius norm:  242.8893319153623
Iteration:  1998  ---> Frobenius norm:  242.87942238734902
Iteration:  1999  ---> Frobenius norm:  242.86952810508072


In [122]:
finalFilledMatrixBias = np.matmul(UBias, VBias.transpose())
finalFilledMatrixBias

array([[4.25803878, 4.57709919, 4.16960205, ..., 4.63062885, 3.89386816,
        4.44215358],
       [3.64108761, 3.96804539, 3.61016848, ..., 4.00540076, 3.26297098,
        3.84864433],
       [3.21241839, 3.53799195, 3.11137203, ..., 3.53175833, 2.76639419,
        3.3730391 ],
       ...,
       [3.92692277, 4.25820406, 3.89206093, ..., 4.28790605, 3.57469105,
        4.12012693],
       [3.45519304, 3.75533368, 3.40688387, ..., 3.83277584, 3.11808434,
        3.65637717],
       [3.92878814, 4.25059136, 3.90485287, ..., 4.29062661, 3.56072525,
        4.13149464]])

In [123]:
errorBatchBias = testMatrix-finalFilledMatrixBias
errorBatchBias[where_are_NaNs] = 0
np.linalg.norm(errorBatchBias)

934.8892035330756